El objetivo de este proyecto es sintetizar automáticamente una función lógica para controlar el segmento 'a' de un display de 7 segmentos, a partir de un número binario de 4 bits que representa un dígito decimal del 0 al 9.

Diseñar la lógica que controla cada segmento puede hacerse manualmente con mapas de Karnaugh, pero aquí se propone un enfoque evolutivo: utilizar Programación Genética (GP) para descubrir automáticamente una expresión lógica equivalente a través de un proceso de selección y evolución.


In [ ]:
!pip install deap



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 10.6 MB/s eta 0:00:00


In [ ]:
from deap import base, creator, gp, tools, algorithms
import operator
import random

# -----------------------------
# Tabla de verdad para el segmento 'a' (decimal → valor segmento 'a')
# Entradas: A, B, C, D (bits de 4)
# Salidas: 1 si el segmento 'a' se enciende, 0 si no
tabla_verdad = [
    ((0, 0, 0, 0), 1),  # 0
    ((0, 0, 0, 1), 0),  # 1
    ((0, 0, 1, 0), 1),  # 2
    ((0, 0, 1, 1), 1),  # 3
    ((0, 1, 0, 0), 0),  # 4
    ((0, 1, 0, 1), 1),  # 5
    ((0, 1, 1, 0), 1),  # 6
    ((0, 1, 1, 1), 1),  # 7
    ((1, 0, 0, 0), 1),  # 8
    ((1, 0, 0, 1), 1),  # 9
]

# -----------------------------
# Operadores lógicos
def AND(a, b): return a & b
def OR(a, b): return a | b
def NOT(a): return ~a & 1
def NAND(a, b): return ~(a & b) & 1
def NOR(a, b): return ~(a | b) & 1
def XOR(a, b): return a ^ b

# -----------------------------
# Configuración del entorno GP
pset = gp.PrimitiveSet("MAIN", 4)
pset.renameArguments(ARG0='A', ARG1='B', ARG2='C', ARG3='D')

pset.addPrimitive(AND, 2)
pset.addPrimitive(OR, 2)
pset.addPrimitive(NOT, 1)
pset.addPrimitive(NAND, 2)
pset.addPrimitive(NOR, 2)
pset.addPrimitive(XOR, 2)

pset.addTerminal(0)
pset.addTerminal(1)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# -----------------------------
# Función de evaluación
def eval_segmento_a(ind):
    func = toolbox.compile(expr=ind)
    aciertos = 0
    for entrada, salida_esperada in tabla_verdad:
        try:
            salida = func(*entrada)
            if salida == salida_esperada:
                aciertos += 1
        except Exception:
            pass
    return aciertos,

toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", eval_segmento_a)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)
toolbox.decorate("mate", gp.staticLimit(key=len, max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=len, max_value=17))

# -----------------------------
# Ejecutar el algoritmo
if __name__ == "__main__":
    random.seed(42)
    poblacion = toolbox.population(n=100)
    hof = tools.HallOfFame(1)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", lambda x: round(sum(v[0] for v in x) / len(x), 2))
    stats.register("max", lambda x: max(v[0] for v in x))

    poblacion, log = algorithms.eaSimple(
        poblacion, toolbox,
        cxpb=0.5, mutpb=0.2,
        ngen=40, stats=stats, halloffame=hof, verbose=True
    )

    print("\n🏆 Mejor individuo (para segmento 'a'):")
    print(hof[0])
    print(f"Aciertos: {hof[0].fitness.values[0]} / 10")


gen	nevals	avg 	max
0  	100   	5.22	8  
1  	55    	6.48	8  
2  	63    	7.15	8  
3  	64    	7.45	8  
4  	52    	7.29	8  
5  	60    	7.44	8  
6  	59    	7.64	8  
7  	56    	7.61	8  
8  	63    	7.64	8  
9  	55    	7.55	8  
10 	60    	7.61	8  
11 	58    	7.72	8  
12 	48    	7.69	8  
13 	75    	7.5 	8  
14 	57    	7.63	8  
15 	68    	7.62	8  
16 	60    	7.68	8  
17 	58    	7.76	8  
18 	57    	7.83	8  
19 	59    	7.86	8  
20 	48    	7.86	8  
21 	56    	7.87	8  
22 	64    	7.75	9  
23 	57    	7.85	9  
24 	56    	7.91	9  
25 	58    	7.88	9  
26 	61    	7.87	9  
27 	58    	8.18	9  
28 	65    	8.22	9  
29 	60    	8.2 	9  
30 	58    	8.42	9  
31 	57    	8.38	9  
32 	57    	8.48	9  
33 	55    	8.35	9  
34 	52    	8.41	9  
35 	68    	8.51	9  
36 	57    	8.61	9  
37 	56    	8.54	9  
38 	59    	8.49	9  
39 	63    	8.4 	9  
40 	69    	8.39	9  

🏆 Mejor individuo (para segmento 'a'):
NAND(AND(OR(0, NOR(C, D)), NOR(A, A)), B)
Aciertos: 9.0 / 10
